### Sample Selection for Galaxies

In [6]:
from astroquery.ned import Ned
import astropy.units as u
from astropy import coordinates

from astropy.table import Table
import pickle

import math

### Global parameters############################################
DDec = 360. / 3600.  ## step for Dec in deg
DRa  = 360. / 3600   ## Step for Ra in deg (bias with projection)

RAmin    = 22. * 360. / 24.       ## in Deg.
RAmax    = 23. * 360. / 24.      ## in Deg.
Decmin   = -50.0                  ## in Deg.
Decmax   = -10.0                  ## in Deg.

Velmin   = 500.
Velmax   = 1500.

sampleGal = []
#################################################################

In [7]:
def isInSample(line):
    "Check if already in sample"
    
    isFound = False
    
    for gal in sampleGal:
        if line[1] == gal['Name'] :
            isFound = True
            
    return(isFound)

In [8]:
def query_NED(co, vmin,vmax):
    "Do a conesearch on NED an return name, redshift, type"
    
    size = 0.1 * u.deg
    try:
        res = Ned.query_region(co, radius=  size , equinox='J2000.0')
    except:
        res = []
    
    galFound = 0
    
    for line in res:
        name = line[1]
        coord = [line[2],line[3]]
        vel  = float(line['Velocity'])
        type = line['Type']
        
        
        if vel > vmin and vel < vmax and not isInSample(line):
            newGal = {}
            newGal['Name'] = name
            newGal['Vel']  = vel
            newGal['Coord'] = coord
            newGal['Type'] =  type
            

            try:
                res = Ned.get_table(name, table='photometry')
            except:
                res = []
                
            newGal['Photometry'] =  res
            
            print ("Name    : %s"%(name))
            print ("Velocity: %6.1f"%(vel))
            print("--------")
            
            sampleGal.append(newGal)
            
            galFound += 1
            
        
     
    return(galFound)
    


In [9]:
def scanSkyRegion(vmin,vmax):
    """
    Scan a sky area to search for Galaxy candidate
    vmin, vmax: velocity range in km/s
    """
    
    ndec = int((Decmax-Decmin)/DDec)
    nra = int((RAmax-RAmin)/DRa)
    ntotal = float(nra*ndec)
    
    galTotal = 0
    print("Pointing; %d %d"%(nra, ndec))
    
    ncurrent = 0.
    
    for i in range(nra):
        for j in range(ndec):
            dec = Decmin + DDec * j
            ra  = (RAmin + DRa * i) % 360.0
            co = coordinates.SkyCoord(ra, dec, unit=(u.deg, u.deg), frame='fk4')
            ng = query_NED(co, vmin, vmax)
            
            
            galTotal += ng
            ncurrent += 1.0
            
            progress = 100. * ncurrent / ntotal
            
            if progress % 10.0 == 0:
                print("## Progress : %3.0f %%"%(progress))
            
    print("## Gal. found: %d"%(galTotal))
            
    

In [10]:
### Run the scanning on NED

print("## Running the scan on the area ...")

scanSkyRegion(Velmin,Velmax)

fileout = 'sampleGalTest.pkl'
f = open(fileout,'w')
pickle.dump(sampleGal,f)
f.close()

print("## Scanning done.")

## Running the scan on the area ...
Pointing; 150 400


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:15: UserWarning: Warning: converting a masked element to nan.


Name    : ESO 288-IG 042
Velocity: 1394.0
--------
## Progress :  10 %
## Progress :  20 %
Name    : 2dFGRS S176Z065
Velocity:  866.0
--------
Name    : 6dF J2218489-461303
Velocity: 1208.0
--------
Name    : IC 5201
Velocity:  915.0
--------
Name    : ESO 289- G 020
Velocity:  911.0
--------
## Progress :  30 %
Name    : [TSK2008] 0352
Velocity:  905.0
--------
Name    : ESO 238- G 005
Velocity:  706.0
--------
Name    : GALEXASC J222324.96-281234.7
Velocity: 1027.0
--------
Name    : 2MASX J22263365-2917276
Velocity: 1025.0
--------
## Progress :  40 %
## Progress :  50 %
Name    : ESO 289-IG 037
Velocity: 1261.0
--------
Name    : 2MASS J22330139-3721347
Velocity:  964.0
--------
Name    : GALEXASC J223329.24-372730.3
Velocity: 1008.0
--------
Name    : 2MASX J22350531-4627406
Velocity:  936.0
--------
Name    : AM 2232-394
Velocity:  909.0
--------
Name    : 2dFGRS S120Z039
Velocity: 1469.0
--------
Name    : [TSK2008] 1129
Velocity: 1430.0
--------
Name    : NGC 7314
Velocity: 142